# Notes

Fourth iteration of the modeling:
- modulirized data processing
- model serialization

In [1]:
import os 
import re
import csv
import json
import math
import random
import time

from itertools import chain
from datetime import datetime

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

# Constants

In [2]:
dir_root = os.path.join('..')
dir_data_root = os.path.join(dir_root, 'data')
dir_exercises = os.path.join(dir_data_root, 'json', 'patients')
dir_exercises_augmented = os.path.join(dir_data_root, 'json', 'patients_augmented')
dir_exercises_test = os.path.join(dir_data_root, 'json', 'test')

**TODO** Implement augmenation and patients sessions for the new data set

# Data prep

### Build training set

In [3]:
CONFIG = {
    'SEQ_MAX_LEN': 801,
    'NUM_EXERCISES': 9,
    'NUM_CLASSES': 6,
    'REGIONS':json.load(open(os.path.join('settings', 'pois_region_tpcnn.json'))),
    'AUGMENTED': True
}

In [4]:
CONFIG

{'SEQ_MAX_LEN': 801,
 'NUM_EXERCISES': 9,
 'NUM_CLASSES': 6,
 'REGIONS': {'LefteyeMidbottom': 'upper',
  'LefteyeMidtop': 'upper',
  'LefteyeInnercorner': 'upper',
  'LefteyeOutercorner': 'upper',
  'LefteyebrowInner': 'upper',
  'LefteyebrowCenter': 'upper',
  'RighteyeMidbottom': 'upper',
  'RighteyeMidtop': 'upper',
  'RighteyeInnercorner': 'upper',
  'RighteyeOutercorner': 'upper',
  'RighteyebrowInner': 'upper',
  'RighteyebrowCenter': 'upper',
  'NoseTip': 'lower',
  'MouthLowerlipMidbottom': 'lower',
  'MouthLeftcorner': 'lower',
  'MouthRightcorner': 'lower',
  'MouthUpperlipMidtop': 'lower',
  'ChinCenter': 'lower',
  'ForeheadCenter': 'upper',
  'LeftcheekCenter': 'lower',
  'RightcheekCenter': 'lower'},
 'AUGMENTED': True}

input for global region
801 is length of a series in excercise 
each series consist from 63 value, 21 POIs, each POI has 3 values: x,y,z

overall length of a series is 7209 (in paper 8019?)

In [5]:
i = 0 
sessions_id = set()
for file_name in os.listdir(dir_exercises):
    ids = file_name.split('_')
    session_id = f'{ids[0]}_{ids[1]}'
    sessions_id.add(session_id)
sessions_ids = sorted(list(sessions_id))

In [6]:
len(sessions_ids)

292

In [7]:
def pad_excercise_poi(data, poi):
    return tf.keras.preprocessing.sequence.pad_sequences(
        [
            data['pois'][poi]['xs'],
            data['pois'][poi]['ys'],
            data['pois'][poi]['zs']
        
        ],
        padding='pre',
        maxlen=CONFIG['SEQ_MAX_LEN'],
        dtype='float32')

In [8]:
def session_data(session_id, augmented = None):
    global_region = None
    upper_region = None
    lower_region = None
    exercises_data = []
    
    for exercise_id in range(CONFIG['NUM_EXERCISES']):
        #todo add handling when augmented is defined and number
        if augmented:
            exercises_data.append(json.load(open(os.path.join(dir_exercises_augmented, f'{session_id}_0{exercise_id + 1}_{augmented}.json'))))
        else:
            exercises_data.append(json.load(open(os.path.join(dir_exercises, f'{session_id}_0{exercise_id + 1}.json'))))
    
    for poi in sorted(CONFIG['REGIONS'].keys()):
        poi_series = None
        
        for exercise_id in range(CONFIG['NUM_EXERCISES']):
            exercise_series = pad_excercise_poi(exercises_data[exercise_id], poi)
            if poi_series is not None:
                poi_series = np.hstack((poi_series, exercise_series))
            else:
                poi_series = exercise_series
        
        if global_region is not None:
            global_region = np.concatenate((global_region, poi_series), axis=0)
        else:
            global_region = poi_series
        
        if CONFIG['REGIONS'][poi] == 'upper':
            if upper_region is not None:
                upper_region = np.concatenate((upper_region, poi_series), axis=0)
            else:
                upper_region = poi_series
        if CONFIG['REGIONS'][poi] == 'lower':
            if lower_region is not None:
                lower_region = np.concatenate((lower_region, poi_series), axis=0)
            else:
                lower_region = poi_series

    
    y = int(exercises_data[0]['meta']['evaluation']) - 1
    

    return [
        global_region,
        upper_region,
        lower_region,
        y
    ]

In [9]:
xslist_global = list()
xslist_upper = list()
xslist_lower = list()
yslist = list()
incomlete_sessions = []
i = 0
for session_id in sessions_ids:
    try:
        _xs_global, _xs_upper, _xs_lower, _ys = session_data(session_id)
        

        yslist.append(_ys)
        xslist_global.append(_xs_global)  
        xslist_upper.append(_xs_upper)
        xslist_lower.append(_xs_lower)
    except:
        print(f'issues with session_id: {session_id}')
        incomlete_sessions.append(session_id)
        continue
    
    if CONFIG['AUGMENTED']:
        for augmentation_id in range(4):
            try:
                _xs_global, _xs_upper, _xs_lower, _ys = session_data(session_id, augmentation_id)


                yslist.append(_ys)
                xslist_global.append(_xs_global)  
                xslist_upper.append(_xs_upper)
                xslist_lower.append(_xs_lower)
            except:  
                break
    
ys = np.array(yslist, dtype='int')
xs_global = np.array(xslist_global, dtype='float32') 
xs_upper = np.array(xslist_upper, dtype='float32') 
xs_lower = np.array(xslist_lower, dtype='float32') 


print(ys.shape)
print(xs_global.shape)
print(xs_upper.shape)
print(xs_lower.shape)
print(f'Incomplete sessions {len(incomlete_sessions)}')

issues with session_id: 105_126
issues with session_id: 107_50
issues with session_id: 107_51
issues with session_id: 110_120
issues with session_id: 116_249
issues with session_id: 145_298
issues with session_id: 156_310
issues with session_id: 17_83
issues with session_id: 17_84
issues with session_id: 187_344
issues with session_id: 19_100
issues with session_id: 1_76
issues with session_id: 20_140
issues with session_id: 23_7
issues with session_id: 257_416
issues with session_id: 26_134
issues with session_id: 27_92
issues with session_id: 28_78
issues with session_id: 28_79
issues with session_id: 32_99
issues with session_id: 35_127
issues with session_id: 3_69
issues with session_id: 44_38
issues with session_id: 44_40
issues with session_id: 47_124
issues with session_id: 4_141
issues with session_id: 55_109
issues with session_id: 55_112
issues with session_id: 57_137
issues with session_id: 58_16
issues with session_id: 60_125
issues with session_id: 63_58
issues with sessio

In [10]:
print(ys)

[0 0 0 ... 0 0 0]


In [11]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


## Modeling

In [48]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.utils import class_weight

In [49]:
def get_conv_x(m, n_feature_maps, multiplier):
    m = Conv1D(n_feature_maps*multiplier, 8, 1, padding='same', activation='relu')(m)
    m = BatchNormalization()(m)
    
    return m

def get_conv_y(m, n_feature_maps, multiplier):
    m = Conv1D(n_feature_maps*multiplier, 5, 1, padding='same', activation='relu')(m)
    m = BatchNormalization()(m)
    
    return m

def get_conv_z(m, n_feature_maps, multiplier):
    m = Conv1D(n_feature_maps*multiplier, 3, 1, padding='same', activation='relu')(m)
    m = BatchNormalization()(m)

    
    return m

def merge(m, shortcut, input_shape, n_feature_maps):
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut = Conv1D(n_feature_maps, 1, 1,padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = BatchNormalization()(shortcut)
    
    m = Add()([shortcut, m])
    
    return m

In [50]:
xs_global.shape

(1240, 63, 7209)

In [53]:
def get_model():
    n_feature = 32
    input_shape = xs_global.shape[1:]
    input_layer = Input(shape=input_shape)
    shortcut = input_layer
    
    model = BatchNormalization()(input_layer)
    
    model = get_conv_x(model, n_feature, 1)
    model = get_conv_y(model, n_feature, 1)
    model = get_conv_z(model, n_feature, 1)
    model = merge(model, shortcut, input_shape, n_feature)
    shortcut = model
        
    for i in range(50):
        model = get_conv_x(model, n_feature, 1)
        model = get_conv_y(model, n_feature, 1)
        model = get_conv_z(model, n_feature, 1)
        model = merge(model, shortcut, input_shape, n_feature)
        shortcut = model
    
    model = GlobalAveragePooling1D()(model)
    model= Dense(256, activation="softmax")(model)
    model= Dense(64, activation="softmax")(model)
    model= Dense(64, activation="softmax")(model)
    model= Dense(6, activation="softmax")(model)
        
    model = Model(inputs=input_layer, outputs=model)

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200),
        metrics=['accuracy']
    )
    
    
    return model

In [54]:
test_model = get_model()
test_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 63, 7209)]   0           []                               
                                                                                                  
 batch_normalization_398 (Batch  (None, 63, 7209)    28836       ['input_11[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 conv1d_390 (Conv1D)            (None, 63, 32)       1845536     ['batch_normalization_398[0][0]']
                                                                                                  
 batch_normalization_399 (Batch  (None, 63, 32)      128         ['conv1d_390[0][0]']       

                                                                                                  
 batch_normalization_412 (Batch  (None, 63, 32)      128         ['conv1d_403[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_405 (Conv1D)            (None, 63, 32)       1056        ['add_98[0][0]']                 
                                                                                                  
 conv1d_404 (Conv1D)            (None, 63, 32)       3104        ['batch_normalization_412[0][0]']
                                                                                                  
 batch_normalization_414 (Batch  (None, 63, 32)      128         ['conv1d_405[0][0]']             
 Normalization)                                                                                   
          

 add_102 (Add)                  (None, 63, 32)       0           ['batch_normalization_426[0][0]',
                                                                  'batch_normalization_425[0][0]']
                                                                                                  
 conv1d_418 (Conv1D)            (None, 63, 32)       8224        ['add_102[0][0]']                
                                                                                                  
 batch_normalization_427 (Batch  (None, 63, 32)      128         ['conv1d_418[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_419 (Conv1D)            (None, 63, 32)       5152        ['batch_normalization_427[0][0]']
                                                                                                  
 batch_nor

                                                                                                  
 conv1d_432 (Conv1D)            (None, 63, 32)       3104        ['batch_normalization_440[0][0]']
                                                                                                  
 batch_normalization_442 (Batch  (None, 63, 32)      128         ['conv1d_433[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_441 (Batch  (None, 63, 32)      128         ['conv1d_432[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_106 (Add)                  (None, 63, 32)       0           ['batch_normalization_442[0][0]',
          

 batch_normalization_455 (Batch  (None, 63, 32)      128         ['conv1d_446[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_447 (Conv1D)            (None, 63, 32)       5152        ['batch_normalization_455[0][0]']
                                                                                                  
 batch_normalization_456 (Batch  (None, 63, 32)      128         ['conv1d_447[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_449 (Conv1D)            (None, 63, 32)       1056        ['add_109[0][0]']                
                                                                                                  
 conv1d_44

...

 batch_normalization_541 (Batch  (None, 63, 32)      128         ['conv1d_532[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_131 (Add)                  (None, 63, 32)       0           ['batch_normalization_542[0][0]',
                                                                  'batch_normalization_541[0][0]']
                                                                                                  
 conv1d_534 (Conv1D)            (None, 63, 32)       8224        ['add_131[0][0]']                
                                                                                                  
 batch_normalization_543 (Batch  (None, 63, 32)      128         ['conv1d_534[0][0]']             
 Normalization)                                                                                   
          

 Normalization)                                                                                   
                                                                                                  
 conv1d_549 (Conv1D)            (None, 63, 32)       1056        ['add_134[0][0]']                
                                                                                                  
 conv1d_548 (Conv1D)            (None, 63, 32)       3104        ['batch_normalization_556[0][0]']
                                                                                                  
 batch_normalization_558 (Batch  (None, 63, 32)      128         ['conv1d_549[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_557 (Batch  (None, 63, 32)      128         ['conv1d_548[0][0]']             
 Normaliza

                                                                                                  
 conv1d_562 (Conv1D)            (None, 63, 32)       8224        ['add_138[0][0]']                
                                                                                                  
 batch_normalization_571 (Batch  (None, 63, 32)      128         ['conv1d_562[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_563 (Conv1D)            (None, 63, 32)       5152        ['batch_normalization_571[0][0]']
                                                                                                  
 batch_normalization_572 (Batch  (None, 63, 32)      128         ['conv1d_563[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 batch_normalization_586 (Batch  (None, 63, 32)      128         ['conv1d_577[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_585 (Batch  (None, 63, 32)      128         ['conv1d_576[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_142 (Add)                  (None, 63, 32)       0           ['batch_normalization_586[0][0]',
                                                                  'batch_normalization_585[0][0]']
                                                                                                  
 conv1d_57

                                                                                                  
 conv1d_591 (Conv1D)            (None, 63, 32)       5152        ['batch_normalization_599[0][0]']
                                                                                                  
 batch_normalization_600 (Batch  (None, 63, 32)      128         ['conv1d_591[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv1d_593 (Conv1D)            (None, 63, 32)       1056        ['add_145[0][0]']                
                                                                                                  
 conv1d_592 (Conv1D)            (None, 63, 32)       3104        ['batch_normalization_600[0][0]']
                                                                                                  
 batch_nor

In [55]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


### K-fold evaluation

In [56]:
k = 5
k_limit = 5
train = 0.8
val = 0.2
test = 0.2

In [57]:
VERBOSE = 0
EPOCHS = 400
BATCH_SIZE = 64

In [58]:
from sklearn.model_selection import KFold

In [59]:
def get_k_indx(k, n):

    k_fold = KFold(n_splits=k)
    train_ = []
    val_ = []
    test_ = []
    indx = []

    for train_indices, test_indices in k_fold.split(ys):
        n_k = len(train_indices)
        val_split = int(n_k * train)
        indx.append([train_indices[:val_split],train_indices[val_split + 1:], test_indices])
    
    return indx

In [61]:
%matplotlib inline

In [ ]:
indxs = get_k_indx(k, len(ys))
models = []

fig, axs = plt.subplots(1,5,figsize=(10,2)) 
colors = ['red', 'pink', 'blue', 'cyan']
labels = ['Loss training', 'Loss validation', 'Accuracy training', 'Accuracy validation']

for i in range(k_limit):
    train_indx, val_indx, test_indx  = indxs[i]
    
    xs_global_i = xs_global[train_indx]
    xs_global_i_val = xs_global[val_indx]
    xs_global_i_test = xs_global[test_indx]
    
    ys_i = ys[train_indx]
    ys_i_val = ys[val_indx]
    ys_i_test = ys[test_indx]
    
    
    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ys_i),
                                                 y=ys_i)
    available_classes = np.unique(ys_i)
    weight_training_classes = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
    }
    
    for indx, value in enumerate(class_weights):
        weight_training_classes[available_classes[indx]] = value

    #print(weight_training_classes)

    model = get_model()

    model_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20)
    ]
    
    hist = model.fit(
        x=xs_global_i, y=ys_i, 
        validation_data=(xs_global_i_val, ys_i_val),
        batch_size=BATCH_SIZE, 
        epochs=EPOCHS,
        class_weight=weight_training_classes,
        #callbacks=model_callbacks,
        verbose=VERBOSE)
    
    models.append(model)
    
    # visualizing losses and accuracy
    train_loss = hist.history['loss']
    val_loss   = hist.history['val_loss']
    train_acc  = hist.history['accuracy']
    val_acc    = hist.history['val_accuracy']
    xc         = range(len(hist.history['loss']))    
    
    axs[i].set_title(f'Run {i}')
    axs[i].plot(xc, train_loss, color=colors[0], label=labels[0])
    axs[i].plot(xc, val_loss, color=colors[1], label=labels[1])
    axs[i].plot(xc, train_acc, color=colors[2], label=labels[2])
    axs[i].plot(xc, val_acc, color=colors[3], label=labels[3])
    
    y_pred = model.predict(xs_global_i_test,verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(classification_report(ys_i_test, y_pred_bool))
    
    
fig.legend(labels, ncol=1, bbox_to_anchor=(1.2,0.5), loc="center right", fontsize=10, 
           bbox_transform=plt.gcf().transFigure)

plt.show()
fig.savefig(f'resnet_sessions_runs_01-{int(time.time())}.svg', format='svg', dpi=1200)   

2024-03-03 10:22:56.203940: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-03 10:23:28.215056: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 10:33:49.636670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 15:13:53.678424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alanchen/.local/share/v

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       103
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        30
           5       0.12      0.71      0.21        35

    accuracy                           0.10       248
   macro avg       0.02      0.12      0.03       248
weighted avg       0.02      0.10      0.03       248



2024-03-03 15:16:12.580176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 15:38:00.509544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 19:26:35.827270: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        20
           4       0.15      1.00      0.27        10
           5       0.16      0.50      0.25        60

    accuracy                           0.16       248
   macro avg       0.05      0.25      0.09       248
weighted avg       0.05      0.16      0.07       248



# Model Serialization 

In [ ]:
serialize = False
best_model = 2

In [ ]:
dir_models = os.path.join(dir_root, 'models')
dir_model = os.path.join(dir_models, 'type4-with-fe_no-aug_005')

In [ ]:
if serialize:
    model = models[best_model]
    model.save(dir_model)